# Components Of LlamaIndex

In this notebook we will demonstrate building RAG application and customize it using different components of LlamaIndex.

1. Question Answering
2. Summarization.
3. ChatEngine.
4. Customizing QA System.
5. Index as Retriever.

[ChatEngine Documentation](https://docs.llamaindex.ai/en/stable/module_guides/deploying/chat_engines/usage_pattern/ )

## Installation

In [ ]:
!pip install llama-index
# !pip install llama-index-llms-openai
# !pip install llama-index-embeddings-openai

## Setup API Key

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "sk-..."

## Download Data

In [ ]:
!wget "https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt" "paul_graham_essay.txt"

--2024-04-28 04:34:09--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75042 (73K) [text/plain]
Saving to: ‘paul_graham_essay.txt’

paul_graham_essay.t 100%[===================>]  73.28K  --.-KB/s    in 0.01s   

2024-04-28 04:34:09 (5.45 MB/s) - ‘paul_graham_essay.txt’ saved [75042/75042]

--2024-04-28 04:34:09--  http://paul_graham_essay.txt/
Resolving paul_graham_essay.txt (paul_graham_essay.txt)... failed: Name or service not known.
wget: unable to resolve host address ‘paul_graham_essay.txt’
FINISHED --2024-04-28 04:34:09--
Total wall clock time: 0.2s
Downloaded: 1 files, 73K in 0.01s (5.45 MB/s)


## Load Data

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["paul_graham_essay.txt"]
).load_data()

## Set LLM and Embedding Model

In [ ]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.2)
embed_model = OpenAIEmbedding()

Settings.llm = llm
Settings.embed_model = embed_model

## Create Nodes

In [ ]:
from llama_index.core.node_parser import TokenTextSplitter

splitter = TokenTextSplitter(chunk_size=1024, chunk_overlap=20)
nodes = splitter.get_nodes_from_documents(documents)

In [ ]:
nodes[0]

TextNode(id_='957f1fe7-c30c-4ad4-a11d-5abd8df895e0', embedding=None, metadata={'file_path': 'paul_graham_essay.txt', 'file_name': 'paul_graham_essay.txt', 'file_type': 'text/plain', 'file_size': 75042, 'creation_date': '2024-04-28', 'last_modified_date': '2024-04-28'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='bad72a51-4dbd-408a-8049-d86b67cd2077', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': 'paul_graham_essay.txt', 'file_name': 'paul_graham_essay.txt', 'file_type': 'text/plain', 'file_size': 75042, 'creation_date': '2024-04-28', 'last_modified_date': '2024-04-28'}, hash='6faa426d61b507e88eb0c1e1e16b50d553ae2a88d0ea43b14f7b0323e893b6c9'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(

## Create Index

In [ ]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex(nodes)  # VectorStoreIndex.from_documents(documents)

## Create QueryEngine

In [ ]:
query_engine = index.as_query_engine(similarity_top_k=5)

## Querying

In [ ]:
response = query_engine.query("What did Paul Graham do growing up?")

In [ ]:
print(response)

Growing up, Paul Graham wrote short stories and dabbled in programming. He started with an IBM 1401 in 9th grade, using an early version of Fortran. Transitioning to microcomputers like the TRS-80, he wrote simple games, a rocket prediction program, and a word processor that his father used to write a book.


In [ ]:
print(len(response.source_nodes))

5


In [ ]:
response.source_nodes[0]

NodeWithScore(node=TextNode(id_='9ba45177-a374-48d4-87b5-9beb6e32bb6d', embedding=None, metadata={'file_path': 'paul_graham_essay.txt', 'file_name': 'paul_graham_essay.txt', 'file_type': 'text/plain', 'file_size': 75042, 'creation_date': '2024-04-28', 'last_modified_date': '2024-04-28'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='bad72a51-4dbd-408a-8049-d86b67cd2077', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': 'paul_graham_essay.txt', 'file_name': 'paul_graham_essay.txt', 'file_type': 'text/plain', 'file_size': 75042, 'creation_date': '2024-04-28', 'last_modified_date': '2024-04-28'}, hash='6faa426d61b507e88eb0c1e1e16b50d553ae2a88d0ea43b14f7b0323e893b6c9'), <NodeRelationship.PREVIOUS:

## Summarization

In [ ]:
from llama_index.core import SummaryIndex

summary_index = SummaryIndex(nodes)

query_engine = summary_index.as_query_engine()

summary = query_engine.query("Provide the summary of the document.")
print(summary)

The document chronicles the author's journey through different endeavors, such as writing, programming, exploring artificial intelligence, Lisp programming, art, and painting still life.


## ChatEngines

### Simple ChatEngine

In [ ]:
from llama_index.core.chat_engine import SimpleChatEngine

chat_engine = SimpleChatEngine.from_defaults()

In [ ]:
response = chat_engine.chat("Hello")
response

AgentChatResponse(response='Hello! How can I assist you today?', sources=[], source_nodes=[], is_dummy_stream=False)

In [ ]:
response = chat_engine.chat("What did steve jobs do growing up?")
response

AgentChatResponse(response='Steve Jobs was born on February 24, 1955, and grew up in California. As a child, he showed an early interest in electronics and technology. He attended Reed College but dropped out after just one semester. Jobs then went on to work at Atari before co-founding Apple Inc. in 1976 with Steve Wozniak. Throughout his career, Jobs was known for his innovative ideas, design aesthetics, and visionary leadership in the tech industry.', sources=[], source_nodes=[], is_dummy_stream=False)

In [ ]:
response = chat_engine.chat("And did he visit India?")
response

AgentChatResponse(response='Yes, Steve Jobs did visit India in the 1970s. After dropping out of college, Jobs traveled to India in search of spiritual enlightenment. He spent several months in India, visiting various ashrams and seeking guidance from spiritual teachers. This experience had a profound impact on Jobs and influenced his personal philosophy and approach to life and work. Jobs later credited his time in India as a significant influence on his thinking and creativity.', sources=[], source_nodes=[], is_dummy_stream=False)

In [ ]:
chat_engine.chat_repl()

===== Entering Chat REPL =====
Type "exit" to exit.

Human: hi
Assistant: Hello! How can I assist you today?

Human: who is steve jobs
Assistant: Steve Jobs was an American entrepreneur, business magnate, and inventor who co-founded Apple Inc. He is widely regarded as a pioneer of the personal computer revolution and played a key role in the development of iconic products such as the iPhone, iPad, and Macintosh computer. Jobs was known for his visionary leadership, innovative design aesthetic, and relentless pursuit of excellence. He passed away in 2011, but his legacy continues to inspire and influence the tech industry and beyond.

Human: exit


### CondenseQuestion ChatEngine

In [ ]:
chat_engine = index.as_chat_engine(chat_mode="condense_question", verbose=True)

In [ ]:
response = chat_engine.chat("What did Paul Graham do after YC?")

Querying with: What did Paul Graham do after YC?


In [ ]:
print(response)

Paul Graham started working on a new version of Arc with Robert in the summer of 2006. This version of Arc was compiled into Scheme, and to test it, Paul wrote Hacker News. Initially meant to be a news aggregator for startup founders, it was later renamed Hacker News with a broader topic to engage intellectual curiosity. Paul gradually stopped working on Arc as he focused more on writing essays and working on Y Combinator.


In [ ]:
response = chat_engine.chat("What about after that?")

Querying with: What did Paul Graham do after working on Arc, Hacker News, and writing essays for Y Combinator?


In [ ]:
print(response)

Paul Graham continued to work on Y Combinator after working on Arc, Hacker News, and writing essays.


In [ ]:
response = chat_engine.chat("Can you tell me more?")

Querying with: Can you tell me more about what Paul Graham did after working on Arc, Hacker News, and writing essays?


In [ ]:
print(response)

Paul Graham continued his journey by collaborating with Jessica Livingston to start their own investment firm after realizing the need to implement their ideas without waiting for venture capitalists to make decisions.


### Context ChatEngine

In [ ]:
from llama_index.core.memory import ChatMemoryBuffer

memory = ChatMemoryBuffer.from_defaults(token_limit=3900)

chat_engine = index.as_chat_engine(
    chat_mode="context",
    memory=memory,
    system_prompt=(
        "You are a chatbot, able to have normal interactions, as well as talk"
        " about an essay discussing Paul Grahams life."
    ),
)

In [ ]:
response = chat_engine.chat("Hello")
print(response)

Hello! How are you doing today?


In [ ]:
response = chat_engine.chat("What did Paul Graham do after YC?")
print(response)

After Y Combinator (YC), Paul Graham continued to be involved in various projects. He worked on a new version of Arc in the summer of 2006 and created Hacker News, which became a popular platform for discussions on various topics. Additionally, he wrote essays and continued to be engaged in the startup community. His work after YC involved a mix of writing, software development, and other entrepreneurial endeavors.


In [ ]:
response = chat_engine.chat("What about after that?")
print(response)

After his time at Y Combinator and his various projects, Paul Graham continued to write essays, share his insights on startups and technology, and engage with the entrepreneurial community. He also co-founded the venture capital firm Y Combinator, which has become one of the most successful startup accelerators in the world. Graham's influence and contributions to the tech industry have continued to be significant, and he remains a respected figure in the startup ecosystem.


In [ ]:
response = chat_engine.chat("Can you tell me more?")
print(response)

Certainly! If you're interested in learning more about Paul Graham's life and his contributions to the tech industry, I can share insights from an essay that discusses various aspects of his journey. Would you like me to provide some excerpts from the essay for more detailed information?


### CondenseContext ChatEngine

In [ ]:
from llama_index.core.memory import ChatMemoryBuffer

memory = ChatMemoryBuffer.from_defaults(token_limit=3900)

chat_engine = index.as_chat_engine(
    chat_mode="condense_plus_context",
    memory=memory,
    llm=llm,
    context_prompt=(
        "You are a chatbot, able to have normal interactions, as well as talk"
        " about an essay discussing Paul Grahams life."
        "Here are the relevant documents for the context:\n"
        "{context_str}"
        "\nInstruction: Use the previous chat history, or the context above, to interact and help the user."
    ),
    verbose=True,
)

In [ ]:
response = chat_engine.chat("Hello")

Condensed question: Hello
Context: file_path: paul_graham_essay.txt

then I thought perhaps it would be interesting to other people, and encouraging to those with similarly messy lives. So I wrote a more detailed version for others to read, and this is the last sentence of it.









Notes

[1] My experience skipped a step in the evolution of computers: time-sharing machines with interactive OSes. I went straight from batch processing to microcomputers, which made microcomputers seem all the more exciting.

[2] Italian words for abstract concepts can nearly always be predicted from their English cognates (except for occasional traps like polluzione). It's the everyday words that differ. So if you string together a lot of abstract concepts with a few simple verbs, you can make a little Italian go a long way.

[3] I lived at Piazza San Felice 4, so my walk to the Accademia went straight down the spine of old Florence: past the Pitti, across the bridge, past Orsanmichele, between the Du

In [ ]:
print(response)

Hello! How are you today?


In [ ]:
response = chat_engine.chat("What did Paul Graham do after YC?")

Condensed question: What did Paul Graham do after Y Combinator?
Context: file_path: paul_graham_essay.txt

so long to make up their minds. We'd start our own investment firm and actually implement the ideas we'd been talking about. I'd fund it, and Jessica could quit her job and work for it, and we'd get Robert and Trevor as partners too. [13]

Once again, ignorance worked in our favor. We had no idea how to be angel investors, and in Boston in 2005 there were no Ron Conways to learn from. So we just made what seemed like the obvious choices, and some of the things we did turned out to be novel.

There are multiple components to Y Combinator, and we didn't figure them all out at once. The part we got first was to be an angel firm. In those days, those two words didn't go together. There were VC firms, which were organized companies with people whose job it was to make investments, but they only did big, million dollar investments. And there were angels, who did smaller investments, but

In [ ]:
print(response)

After Y Combinator (YC), Paul Graham continued to focus on writing essays and working on various projects. He also worked on a new version of Arc, a programming language he created, and developed Hacker News, a news aggregator platform. Additionally, he remained involved in the startup community and continued to offer advice and insights through his writings and public appearances.


In [ ]:
response = chat_engine.chat("What about after that?")

Condensed question: What did Paul Graham do after his involvement with Y Combinator, writing essays, working on projects, developing Arc, and creating Hacker News?
Context: file_path: paul_graham_essay.txt

which in the typical two-founder case was $12k, in return for 6%. That had to be fair, because it was twice as good as the deal we ourselves had taken. Plus that first summer, which was really hot, Jessica brought the founders free air conditioners. [16]

Fairly quickly I realized that we had stumbled upon the way to scale startup funding. Funding startups in batches was more convenient for us, because it meant we could do things for a lot of startups at once, but being part of a batch was better for the startups too. It solved one of the biggest problems faced by founders: the isolation. Now you not only had colleagues, but colleagues who understood the problems you were facing and could tell you how they were solving them.

As YC grew, we started to notice other advantages of scal

In [ ]:
print(response)

After his involvement with Y Combinator and working on projects like Arc and Hacker News, Paul Graham continued to write essays and share his thoughts on various topics. He also remained active in the tech and startup community, offering advice and insights based on his experiences. Additionally, he continued to explore new ideas and projects, always looking for ways to contribute to the world of technology and entrepreneurship.


In [ ]:
response = chat_engine.chat("Can you tell me more?")

Condensed question: Tell me more about Paul Graham's activities after his involvement with Y Combinator and his projects like Arc and Hacker News.
Context: file_path: paul_graham_essay.txt

which in the typical two-founder case was $12k, in return for 6%. That had to be fair, because it was twice as good as the deal we ourselves had taken. Plus that first summer, which was really hot, Jessica brought the founders free air conditioners. [16]

Fairly quickly I realized that we had stumbled upon the way to scale startup funding. Funding startups in batches was more convenient for us, because it meant we could do things for a lot of startups at once, but being part of a batch was better for the startups too. It solved one of the biggest problems faced by founders: the isolation. Now you not only had colleagues, but colleagues who understood the problems you were facing and could tell you how they were solving them.

As YC grew, we started to notice other advantages of scale. The alumni bec

In [ ]:
print(response)

Certainly! Paul Graham is known for his contributions to the tech industry, particularly through his work with Y Combinator, a startup accelerator he co-founded. He has written numerous essays on a wide range of topics, including startups, programming, and entrepreneurship. Graham's insights and advice have been valuable to many aspiring entrepreneurs and developers.

In addition to his writing and work with startups, Graham has also been involved in creating programming languages like Arc and developing platforms such as Hacker News. His passion for technology and innovation is evident in his various projects and endeavors over the years.

Overall, Paul Graham's impact on the tech community extends beyond his time at Y Combinator, as he continues to inspire and educate through his writings and contributions to the field of technology and entrepreneurship.


## Customizing RAG Pipeline

In [ ]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.response_synthesizers import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine

# create nodes
splitter = TokenTextSplitter(chunk_size=1024, chunk_overlap=20)
nodes = splitter.get_nodes_from_documents(documents)

# Construct an index by loading documents into a VectorStoreIndex.
index = VectorStoreIndex(nodes)

# configure retriever
retriever = VectorIndexRetriever(index=index, similarity_top_k=3)

# configure response synthesizer
synthesizer = get_response_synthesizer(response_mode="refine")

# construct query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=synthesizer,
)

In [ ]:
response = query_engine.query("What did Paul Graham do growing up?")

In [ ]:
print(response)

Paul Graham grew up to become a successful entrepreneur and programmer.


## Index as Retriever

In [ ]:
retriever = index.as_retriever(similarity_top_k=3)

In [ ]:
retrieved_nodes = retriever.retrieve("What did Paul Graham do growing up?")

In [ ]:
from llama_index.core.response.notebook_utils import display_source_node

for text_node in retrieved_nodes:
    display_source_node(text_node, source_length=500)

**Node ID:** 83fb80c2-eef6-4f32-b632-f1ed4034cf15<br>**Similarity:** 0.8332996078161921<br>**Text:** I decided to move to Cambridge and start it. I hoped to lure Robert into working on it with me, but there I ran into a hitch. Robert was now a postdoc at MIT, and though he'd made a lot of money the last time I'd lured him into working on one of my schemes, it had also been a huge time sink. So while he agreed that it sounded like a plausible idea, he firmly refused to work on it.

Hmph. Well, I'd do it myself then. I recruited Dan Giffin, who had worked for Viaweb, and two undergrads who wan...<br>

**Node ID:** 4a367ce5-0b22-4377-9341-fc9285d065a0<br>**Similarity:** 0.8268013589103509<br>**Text:** CMU, where he was in grad school. One day I went to visit the Carnegie Institute, where I'd spent a lot of time as a kid. While looking at a painting there I realized something that might seem obvious, but was a big surprise to me. There, right on the wall, was something you could make that would last. Paintings didn't become obsolete. Some of the best ones were hundreds of years old.

And moreover this was something you could make a living doing. Not as easily as you could by writing softwar...<br>

**Node ID:** 223787cc-0e88-40bd-94b7-b7003fcaffea<br>**Similarity:** 0.8230760740448807<br>**Text:** paint, and I could do that for free. So in 1993 I dropped out. I hung around Providence for a bit, and then my college friend Nancy Parmet did me a big favor. A rent-controlled apartment in a building her mother owned in New York was becoming vacant. Did I want it? It wasn't much more than my current place, and New York was supposed to be where the artists were. So yes, I wanted it! [7]

Asterix comics begin by zooming in on a tiny corner of Roman Gaul that turns out not to be controlled by t...<br>